In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('2673665.answers.csv')

In [ ]:
df.head()

In [ ]:
df['student_response']

In [ ]:
isinstance(1.0,int)

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ["a", "an", "the", "this", "that", "is", "it", "to", "and"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text_stemming(texts): 
    final_text_list=[]
    for sent in texts:
        filtered_sentence=[]
        
        #sent = ''.join([i for i in sent if not i.isdigit()])
        if not isinstance(sent,float):
            sent = sent.lower() # Lowercase 
            sent = sent.strip() # Remove leading/trailing whitespace
            sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
            sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        
            for w in word_tokenize(sent):
                # We are applying some custom filtering here, feel free to try different things
                # Check if it is not numeric and its length>2 and not in stop words
                if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):  
                    # Stem and add to filtered list
                    filtered_sentence.append(snow.stem(w))
            final_string = " ".join(filtered_sentence) #final string of cleaned words

            final_text_list.append(final_string)
    
    return final_text_list

In [ ]:
process_text_stemming(df.iloc[:,0])

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ["a", "an", "the", "this", "that", "is", "it", "to", "and"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
# Full list is available here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def process_text_lemmitization(texts): 
    final_text_list=[]
    for sent in texts:
        filtered_sentence=[]
        
        if not isinstance(sent,float):
            sent = sent.lower() # Lowercase 
            sent = sent.strip() # Remove leading/trailing whitespace
            sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
            sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:

            for w in word_tokenize(sent):
                # We are applying some custom filtering here, feel free to try different things
                # Check if it is not numeric and its length>2 and not in stop words
                if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):  
                    # Stem and add to filtered list
                      filtered_sentence.append(w)

            lemmatized_sentence = []

            # Get position tags
            word_pos_tags = nltk.pos_tag(filtered_sentence)

            # Map the position tag and lemmatize the word/token
            for idx, tag in enumerate(word_pos_tags):
                lemmatized_sentence.append(wl.lemmatize(tag[0], get_wordnet_pos(tag[1])))

            lemmatized_text = " ".join(lemmatized_sentence)
            
            final_text_list.append(lemmatized_text)
    
    return final_text_list

In [ ]:
df_processed = process_text_lemmitization(df['student_response'])

In [ ]:
len(df_processed)

In [ ]:
df_final = pd.DataFrame(columns=['student_response'])

In [ ]:
df_final['student_response'] = df_processed

In [ ]:
df_final.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Initialize the binary count vectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf=True,
                                max_features=50    # Limit the vocabulary size
                                )
# Fit and transform
text_vectors = tfidf_vectorizer.fit_transform(df_processed)

In [ ]:
text_vectors.toarray()

In [ ]:
tfidf_vectorizer.vocabulary_

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=5).fit(text_vectors)

In [ ]:
kmeans

In [ ]:
res = kmeans.predict(text_vectors)

In [ ]:
len(res)

In [ ]:
df_final.shape

In [ ]:
import numpy as np

In [ ]:
results = df_final

In [ ]:
results['cluster'] = res

In [ ]:
results.head()

In [ ]:
results.to_csv('results.csv',index=False)